In [36]:
#!pip install -U transformers
#!pip install -q gradio

In [37]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel
import gradio as gr

In [38]:
#tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
#model = GPT2LMHeadModel.from_pretrained("gpt2", pad_token_id=tokenizer.eos_token_id)

In [39]:
import torch
from transformers import LEDTokenizer, LEDForConditionalGeneration
tokenizer = LEDTokenizer.from_pretrained("bakhitovd/led-base-7168-ml")
model = LEDForConditionalGeneration.from_pretrained("bakhitovd/led-base-7168-ml")

In [40]:
# Initialize conversation history
conversation_history = []

# Define the text generation function
def generate_text(input_text):
    global conversation_history
    # Append the new input to the conversation history
    conversation_history.append(f"User: {input_text}")
    # Concatenate the conversation history
    context = " ".join(conversation_history)
    input_ids = tokenizer.encode(context, return_tensors='pt')
    
    # Truncate the input_ids if it exceeds the model's maximum length
    max_length = model.config.max_position_embeddings
    if input_ids.shape[1] > max_length:
        input_ids = input_ids[:, -max_length:]
    
    # Generate text with adjusted parameters
    generated_ids = model.generate(
        input_ids, 
        max_length=150,  # Adjusted max_length
        num_beams=5, 
        no_repeat_ngram_size=2, 
        early_stopping=True,
        temperature=0.7,  # Added temperature
        top_p=0.7  # Added top_p
    )
    output = tokenizer.decode(generated_ids[0], skip_special_tokens=True, clean_up_tokenization_spaces=True)
    # Append the model's output to the conversation history
    conversation_history.append(f"AI: {output}")
    return output

# Define a function to reset the conversation history
def reset_history():
    global conversation_history
    conversation_history = []
    return "History reset."

In [41]:
# Create the Gradio interface
with gr.Blocks() as demo:
    gr.Markdown("# Game Story!")
    gr.Markdown("A game where you play a story of your own with the AI")
    
    with gr.Row():
        input_text = gr.Textbox(label="Input Text")
        output_text = gr.Textbox(label="Output Text")
    
    generate_button = gr.Button("Generate")
    reset_button = gr.Button("Reset History")
    
    generate_button.click(generate_text, inputs=input_text, outputs=output_text)
    reset_button.click(reset_history, inputs=[], outputs=output_text)

# Launch the interface
demo.launch()

* Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.
